In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import exists, join as pjoin, realpath
NBROOT = realpath(os.curdir)

import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('PIL').level = logging.INFO # PIL.PngImagePlugin

import sys
sys.path.append(realpath('..'))
sys.path.append(realpath('../../eslope/development/src'))

from glob import glob
import json
from pathlib import Path
from time import time
from urllib.error import HTTPError
from urllib.request import urlopen, urlretrieve
from subprocess import check_call, CalledProcessError

try:
    # like os.system but with live output
    from IPython.utils.process import system
    def check_run(cmd):  # type:ignore
        r = system(cmd)
        if r: raise CalledProcessError(r, cmd)
        return r
except ImportError:
    def check_run(cmd):
        return check_call(cmd, shell=True)

#external
#!pip install shapely
import shapely
from shapely.geometry import mapping, shape, GeometryCollection, Polygon, Point
from shapely.ops import cascaded_union
import numpy as np
from IPython.display import display, Image
from PIL import Image as Img
import mercantile as T
from osgeo import gdal
gdal.UseExceptions()

# own
import mbt_util as M
import bbox
from mbt_util import mbt_merge, mbt_info
from src import img_util as G, geometry as GY, \
    mbt_download as MD, mbt_partial as MP, mbt_pyramid as MY, swisstopo as SS
from src.jpg_quality_pil_magick import get_jpg_quality


In [3]:
%cd /opt/map/mapdata/mbtiles

/home/eoubrayrie/Downloads/dwnmaps/mapdata/mbtiles


In [4]:
names = GY.bbalp_names
paths = {i: name + '.mbtiles' for i, name in names.items()}


In [5]:
M.mbt_info('mch.mbtiles')

'zoom = 9 16 ; n = 280691 * 28 kb/tile q = 95 ; bounds = 4.21875,44.59047,13.35938,48.45835 ; center = 8.78906,46.52441,9 ; format = jpeg ; name = SwissTopo mix-scale'

In [5]:
M.get_meta('mch.mbtiles')

{'name': 'SwissTopo mix-scale',
 'description': 'Switzerland SwissTopo National Map, zoom 9-16. https://map.geo.admin.ch\nUnlike the website, this is a downscaled version when possible (1 zoom level up for each layer).',
 'bounds': '4.21875,44.59047,13.35938,48.45835',
 'format': 'jpeg',
 'type': 'baselayer',
 'center': '8.78906,46.52441,9',
 'minzoom': '9',
 'maxzoom': '12'}

## Z12

Make the low-zoom (6-8 + 9-12) file (111 MB)

In [9]:
mbtch12 = M.cut_zoom('mch.mbtiles', zooms=[9, 10, 11, 12], overwrite=True)

Overwriting  mch-z9-12.mbtiles
z 9 : 104
z 10 : 416
z 11 : 264
z 12 : 2591


In [14]:
M.mbt_merge(mbtch12, '../mobac_atlases/OpenTopoMaps-z68.mbtiles', dest='alps12.mbtiles')

cp mch-z9-12.mbtiles alps12.mbtiles
<<>> mch-z9-12 : zoom = 9 12 ; n = 3375 * 34 kb/tile ; bounds = 4.21875,44.59047,13.35938,48.45835 ; center = 8.78906,46.52441,9 ; format = jpeg ; name = SwissTopo mix-scale
<< ../mobac_atlases/OpenTopoMaps-z68 : zoom = 5 8 ; n = 63 * 44 kb/tile ; bounds = 0.000,40.996,22.456,48.922 ; format = png ; name = Unnamed atlas
>> alps12 : zoom = 5 12 ; n = 3438 ; bounds = 4.21875,44.59047,13.35938,48.45835 ; center = 8.78906,46.52441,9 ; format = jpeg ; name = SwissTopo mix-scale
Meta update {'bounds': '0.0,40.9799,22.5,48.9225',
 'center': '11.25000,44.95120,5',
 'maxzoom': 12,
 'minzoom': 5,
 'type': 'baselayer'}
Created: alps12


In [22]:
wjpg = G.white_jpg(256)
z, x, y = M.lnglat2tms(16, lng=9, lat=46)
M.insert_tiles('alps12.mbtiles', [(z, x, y, wjpg)])
M.set_real_bounds('alps12.mbtiles')
M.mbt_info('alps12.mbtiles')

'zoom = 5 16 ; n = 3439 * 35 kb/tile ; bounds = 0.0,40.9799,22.5,48.9225 ; center = 11.25000,44.95120,5 ; format = jpeg ; name = alps12'

## Z16

In [42]:
for path in glob(f'mch*.mbtiles'):
    print(path[:4],
    tuple(round(n, 2) for n in M.real_bounds(path)[2]),  #bb max (union) of zmax
    tuple(round(n, 2) for n in M.real_bounds(path, strict=True)[2]), # bb min (intersection) of zmax
    tuple(round(n, 2) for n in M.compute_inner_bounds(path)),  # min of zcenter
    tuple(round(n, 2) for n in M.compute_strictest_bounds(path)),  # min of zborders
    #  mbt_info(path),
     sep='\n')

mch8
(6.33, 46.56, 8.44, 47.04)
(6.33, 46.56, 8.44, 47.04)
(6.33, 46.56, 8.44, 47.04)
(6.56, 46.56, 8.44, 47.0)
mch9
(8.44, 46.56, 10.55, 47.04)
(8.44, 46.56, 10.55, 47.04)
(8.44, 46.56, 10.55, 47.04)
(8.44, 46.56, 10.46, 46.95)
mch.
(4.22, 44.59, 13.36, 48.46)
(5.98, 45.8, 10.55, 47.04)
(5.98, 46.25, 10.2, 47.04)
(9.07, 46.54, 9.07, 46.56)
mch5
(5.63, 45.58, 7.73, 46.56)
(5.98, 45.81, 7.73, 46.56)
(5.98, 45.93, 7.73, 46.56)
(7.12, 46.13, 6.79, 46.56)
mch7
(9.14, 45.58, 10.55, 46.56)
(9.14, 45.81, 10.55, 46.56)
(9.14, 46.34, 9.28, 46.56)
(9.14, 46.54, 9.27, 46.56)
mch6
(7.73, 45.58, 9.14, 46.56)
(7.73, 45.8, 9.14, 46.56)
(7.73, 46.14, 9.14, 46.56)
(9.07, 45.93, 9.07, 46.56)


In [9]:
from glob import glob

for path in glob(f'*.mbtiles'):
    print(mbt_info(path), path[:4])

zoom = 9 16 ; n = 64787 * 22 kb/tile ; bounds = 5.625,45.58329,7.03125,46.55886 ; center = 6.32813,46.07108,9 ; format = jpg ; name = frit5-Mont-Blanc-Leman-Cervino-Cogne frit
zoom = 9 16 ; n = 109296 * 25 kb/tile ; bounds = 5.625,45.58329,7.73438,46.55886 ; center = 6.67969,46.07108,9 ; format = jpg ; name = alps5-Mont-Blanc-Leman-Cervino-Cogne alps
zoom = 9 16 ; n = 68648 * 37 kb/tile ; bounds = 4.92188,44.58656,7.73438,45.08904 ; center = 6.32813,44.83780,9 ; format = jpg ; name = alps3-Vercors-Ecrins-Queyras-Cozie alps
zoom = 12 16 ; n = 152742 * 23 kb/tile ; bounds = 6.67969,45.58329,7.73438,46.01222 ; center = 7.20703,45.79776,12 ; format = jpg ; name = Bugianen Bugi
zoom = 9 16 ; n = 64814 * 45 kb/tile ; bounds = 5.625,45.08904,8.4375,45.58329 ; center = 7.03125,45.33616,9 ; format = jpg ; name = alps4-Grenoble-Savoie-Susa-Lanzo-GParadiso alps
zoom = 12 16 ; n = 61535 * 23 kb/tile ; bounds = 6.76758,44.02442,8.17383,44.59047 ; center = 7.47070,44.30744,12 ; format = jpg ; name =

In [15]:
for N in (5,6,7,8,9, 10):
    print(N)
    name, chcpath = names[N], paths[N]
    if not os.path.exists(chcpath):
        continue
        M.cut_to_lnglat('mch.mbtiles', chcpath, GY.bbalps[N])
    M.update_mbt_meta(chcpath, name=name)
    M.set_real_bounds(chcpath)

5
6
7
8
9
10


In [21]:
M.get_meta(paths[9])

{'name': 'alps9-CHCW-Vaud-Luzern-Grimselpass',
 'description': 'Switzerland SwissTopo National Map, zoom 9-16. https://map.geo.admin.ch\nUnlike the website, this is a downscaled version when possible (1 zoom level up for each layer).',
 'bounds': '6.32812,46.55886,8.4375,47.04018',
 'format': 'jpeg',
 'type': 'baselayer',
 'center': '7.38281,46.79952,10',
 'minzoom': '10',
 'maxzoom': '16'}

In [20]:
M.remove_lnglat(f'{paths[9][:-8]}z9.mbtiles', paths[9], zmin=9, zmax=9)

cp alps9-CHCW-Vaud-Luzern-Grimselpassz9.mbtiles alps9-CHCW-Vaud-Luzern-Grimselpass.mbtiles
<<>> alps9-CHCW-Vaud-Luzern-Grimselpassz9 : zoom = 9 16 ; n = 65035 * 24 kb/tile q = 88 ; bounds = 6.32812,46.55886,8.4375,47.04018 ; center = 7.38281,46.79952,9 ; format = jpeg ; name = alps9-CHCW-Vaud-Luzern-Grimselpass
z 9 : removed 3
Meta update {'bounds': '6.32812,46.55886,8.4375,47.04018',
 'center': '7.38281,46.79952,10',
 'maxzoom': 16,
 'minzoom': 10,
 'type': 'baselayer'}


In [47]:
print(M.mbt_info(paths[2]))
M.remove_lnglat(f'{paths[2]}', zmin=9, zmax=9)

zoom = 9 16 ; n = 65536 * 19 kb/tile q = 75 ; bounds = 4.922,43.580,6.328,44.590 ; format = jpg ; name = IGNt2-Digne-Aups-Eguilles-Gap
Backing up source as: alps2-Digne-Aups-Eguilles-Gap.mbtiles
cp alps2-Digne-Aups-Eguilles-Gap.bak.mbtiles alps2-Digne-Aups-Eguilles-Gap.mbtiles
<<>> alps2-Digne-Aups-Eguilles-Gap.bak : zoom = 9 16 ; n = 65536 * 19 kb/tile q = 75 ; bounds = 4.922,43.580,6.328,44.590 ; format = jpg ; name = IGNt2-Digne-Aups-Eguilles-Gap
z 9 : removed 4
Meta update {'bounds': '5.27344,43.58039,6.32813,44.59047',
 'center': '5.80078,44.08543,10',
 'maxzoom': 16,
 'minzoom': 10,
 'type': 'baselayer'}


In [16]:
M.cut_to_lnglat(paths[1], bb=GY.bbalps1z10)

Backing up source as: alps1-Mercantour-Ubaye-Cuneese.mbtiles
10 16 16
z 10: +20 tiles: 530<x<534 650<y<653
z 11: +71 tiles: 1060<x<1068 1300<y<1307
z 12: +292 tiles: 2120<x<2139 2600<y<2615
z 13: +1070 tiles: 4240<x<4279 5200<y<5231
z 14: +4268 tiles: 8480<x<8559 10400<y<10463
z 15: +16982 tiles: 16960<x<17119 20800<y<20927
z 16: +67313 tiles: 33920<x<34239 41600<y<41855
Meta update {'bounds': '6.32812,43.58039,8.08594,44.59047',
 'center': '7.20703,44.08543,10',
 'maxzoom': 16,
 'minzoom': 10,
 'type': 'baselayer'}


In [14]:
M.real_bounds(paths[1], log=print)

real bounds  10 [6.33, 43.58, 8.44, 44.59] 530 650 535 653
real bounds  11 [6.33, 43.58, 7.91, 44.59] 1060 1300 1068 1307
real bounds  12 [6.33, 43.58, 8.17, 44.59] 2120 2600 2140 2615
real bounds  13 [6.33, 43.58, 8.17, 44.59] 4240 5200 4281 5231
real bounds  14 [6.33, 43.58, 8.17, 44.59] 8480 10400 8563 10463
real bounds  15 [6.33, 43.58, 8.17, 44.59] 16960 20800 17127 20927
real bounds  16 [6.33, 43.58, 8.17, 44.59] 33920 41600 34255 41855


(10,
 16,
 LngLatBbox(west=6.328124999999996, south=43.580390855607845, east=8.437500000000014, north=44.590467181308846))

# Kompass

In [19]:
M.real_bounds('../mobac_atlases/Kompass_dolo.mbtiles', log=print)

real bounds  8 [8.44, 45.09, 12.66, 47.99] 134 164 136 166
real bounds  9 [9.14, 45.58, 12.66, 47.99] 269 329 273 333
real bounds  10 [9.14, 45.83, 12.66, 47.75] 538 659 547 666
real bounds  11 [9.14, 45.83, 12.48, 47.75] 1076 1318 1094 1333
real bounds  12 [9.14, 45.83, 12.48, 47.75] 2152 2636 2189 2667
real bounds  13 [9.14, 45.83, 12.48, 47.75] 4304 5272 4379 5335
real bounds  14 [9.14, 45.83, 12.48, 47.75] 8608 10544 8759 10671
real bounds  15 [9.14, 45.83, 12.48, 47.75] 17216 21088 17519 21343


(8,
 15,
 LngLatBbox(west=8.437500000000005, south=45.08903556483102, east=12.656250000000009, north=47.98992166741418))

In [27]:
M.compute_inner_bounds('../mobac_atlases/Kompass_dolo.mbtiles')

LngLatBbox(west=Decimal('8.438'), south=Decimal('45.093'), east=Decimal('12.655'), north=Decimal('47.990')) LngLat(lng=10.5465, lat=46.5415) 8 135 8


LngLatBbox(west=10.37109, south=45.8288, east=12.48047, north=47.5172)

In [34]:
M.compute_strictest_bounds('../mobac_atlases/Kompass_dolo.mbtiles')

LngLatBbox(west=11.95313, south=46.43029, east=11.95312, north=46.67959)

46.316045

'= 71118 * 32 kb/tile ; b'

In [24]:
for path in (
    '../mobac_atlases/Kompass_dolo.mbtiles',
    *glob(f'mch*.mbtiles'),
    *glob(f'Bugianen*.mbtiles'),
    *glob('../mobac_atlases/IGNt*.mbtiles'),
    *glob(f'alps*.mbtiles'),
):
    print(M.mbt_info(path)[16:44], os.path.basename(path)[:-8] )

= 71118 * 32 kb/tile q = 89  Kompass_dolo
= 3375 * 34 kb/tile q = 88 ; mch-z9-12
= 64265 * 29 kb/tile q = 88  mch9
= 280691 * 28 kb/tile q = 95 mch
= 61937 * 28 kb/tile q = 95  mch5
= 28259 * 31 kb/tile q = 88  mch7
= 55846 * 30 kb/tile q = 95  mch6
 = 152742 * 23 kb/tile q = 8 Bugianen
 = 61535 * 23 kb/tile q = 80 Bugianen-calps
= 4949 * 18 kb/tile q = 75 ; IGNt11-alps
= 68832 * 19 kb/tile q = 75  IGNt1-Mercantour-Ubaye-Antibes-Embrun
= 49153 * 20 kb/tile q = 75  IGNt4-Chartreuse-Vanoise Chambery-Bardon
= 62043 * 18 kb/tile q = 75  IGNt5-MontBlanc-Chambery-Evian
= 55476 * 20 kb/tile q = 75  IGNt3-Vercors-Ecrins-Cerces
= 65536 * 19 kb/tile q = 75  IGNt2-Digne-Aups-Eguilles-Gap
= 15393 * 20 kb/tile q = 75  IGNt111-addendum
= 109296 * 25 kb/tile q = 75 alps5-Mont-Blanc-Leman-Cervino-Cogne
= 65035 * 24 kb/tile q = 88  alps8
= 68648 * 37 kb/tile q = 95  alps3-Vercors-Ecrins-Queyras-Cozie
= 64814 * 45 kb/tile q = 99  alps4-Grenoble-Savoie-Susa-Lanzo-GParadiso
= 3439 * 35 kb/tile q = 1 ;  al

In [3]:
%cd $NBROOT

/home/eoubrayrie/mapproj/etopo/topo_merge


In [6]:
!gh release upload 202301 ../../mapdata/mbtiles/alps8.mbtiles

Successfully uploaded 1 asset to 202301


# Merge with bugianen

** need to rename/re-meta all swiss files first to avoid mixups
b = 
M.cut_to_lnglat('Buginanen.mbtile', dest5, GY.bbalps5z10, zmin=10)
